In [12]:
import numpy as np
from numpy.linalg import norm, cholesky, solve, inv
from numpy import matmul, eye

np.set_printoptions(precision=4, suppress=False)

In [13]:
def BC_decomp(A):
    """
    Performs Doolittle LU decomposition of matrix A where:
    - B is lower triangular
    - C is upper triangular with ones on diagonal
    - A = B @ C
    """
    if not isinstance(A, np.ndarray):
        A = np.array(A, dtype=float)
    
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square")

    B = np.zeros((n, n))
    C = np.eye(n, n)
    
    for i in range(n):
        # Calculate B elements
        for j in range(i + 1):
            sum_b = sum(B[i,k] * C[k,j] for k in range(j))
            B[i,j] = A[i,j] - sum_b
            
        # Calculate C elements
        for j in range(i + 1, n):
            sum_c = sum(B[i,k] * C[k,j] for k in range(i))
            C[i,j] = (A[i,j] - sum_c) / B[i,i]
            # C[i,j] = (A[i,j] - sum_c)
            
    return B, C

def solve_bc_system(B: np.ndarray, C: np.ndarray, f: np.ndarray) -> np.ndarray:
    """
    Solve system Ax=f where A=BC using:
    1. By=f -> find y
    2. Cx=y -> find x 
    """
    if not isinstance(B, np.ndarray) or not isinstance(C, np.ndarray):
        raise ValueError("Inputs must be numpy arrays")
        
    n = B.shape[0]
    if B.shape[1] != n or C.shape[0] != n or C.shape[1] != n:
        raise ValueError("Invalid matrix dimensions")

    # Step 1: Solve By = f
    y = np.zeros(n)
    for i in range(n):
        y[i] = f[i]
        for k in range(i):
            y[i] -= B[i,k] * y[k]
        y[i] /= B[i,i]

    # Step 2: Solve Cx = y 
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for k in range(i+1, n):
            x[i] -= C[i,k] * x[k]
        # Note: C diagonal elements are 1
        
    return x

In [14]:
def generate_householder_mat(dim):
    rng = np.random.default_rng(52)

    w = rng.random(dim)
    w = w / norm(w, ord=None) 

    lambdas = rng.random(dim)
    lambda_mat = eye(dim)

    for i in range(dim):
        lambda_mat[i, i] = lambdas[i]
    print(lambdas, "\n", lambda_mat)

    H = eye(dim) - 2 * matmul(w.reshape(dim, -1), w.reshape(-1, dim))
    # print(w.reshape(dim, -1), w.reshape(-1, dim))
    print(H)

    return matmul(matmul(H, lambda_mat), H.T)


size_H = 4
test_hh = generate_householder_mat(size_H)
print(test_hh)


[0.5503 0.9683 0.3981 0.0158] 
 [[0.5503 0.     0.     0.    ]
 [0.     0.9683 0.     0.    ]
 [0.     0.     0.3981 0.    ]
 [0.     0.     0.     0.0158]]
[[ 0.2253 -0.7224 -0.6536 -0.008 ]
 [-0.7224  0.3263 -0.6096 -0.0075]
 [-0.6536 -0.6096  0.4485 -0.0068]
 [-0.008  -0.0075 -0.0068  0.9999]]
[[ 0.7034 -0.1592  0.2287  0.0059]
 [-0.1592  0.5382 -0.0416  0.0024]
 [ 0.2287 -0.0416  0.675   0.006 ]
 [ 0.0059  0.0024  0.006   0.0159]]


In [15]:
# def norm(vec):
#   sum = 0
#   for i in vec:
#     sum += i*i 
#   return sum**(1/2)

In [16]:
def generate_positive_definite_matrix(size: int) -> np.ndarray:
    """
    Generate random symmetric positive definite matrix:
    1. Create random matrix A
    2. Compute A^T * A (always positive semi-definite)
    3. Add diagonal dominance to ensure positive definite
    """
    rng = np.random.default_rng(52)
    
    # Create random matrix
    A = rng.random((size, size))
    
    # Make symmetric positive definite
    matrix = A @ A.T  # Ensures positive semi-definite
    
    # Add to diagonal to ensure positive definite
    matrix += size//2 * eye(size)
    
    return matrix

In [17]:
# mat_size = 3
# mat = generate_positive_definite_matrix(3)

# print(mat)
# mat = np.ndarray((2, 2), dtype=np.int64)


def inverse_iteration(A: np.ndarray, tolerance: np.float64 = 1e-13, max_iter: int = 500):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = cholesky(A, upper=False)
    U = L.T

    B, C = BC_decomp(A)

    A_inv = np.linalg.inv(A)

    # U = np.linalg.cholesky(A, upper=True)

    # print(L @ L.T) # A matr

    for _ in range(max_iter):
        x_norm = norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / x_norm  # +- eigenvec

        # L*y = nu
        # U*x = y
        # y = solve(L, nu)
        # x_next = solve(U, y)
        x_next = solve_bc_system(B, C, nu)
        # x_next = matmul(A_inv, nu)

        sigma_prev = sigma
        sigma = matmul(nu, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance and abs(sigma - sigma_prev) != 0:
            # print(_)
            return nu, 1 / sigma

        sigma_prev = sigma
        x_prev = x_next

    return nu, 1 / sigma


eigvec_1, eigval_1 = inverse_iteration(test_hh)

print(eigval_1, eigvec_1)


0.015837404203920213 [-0.008  -0.0075 -0.0068  0.9999]


In [18]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [19]:
def inverse_iteration_with_shift_one(
    A: np.ndarray, tolerance: np.float64 = 1e-13, max_iter: int = 500
):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = cholesky(A, upper=False)
    U = L.T

    B, C = BC_decomp(A)

    g_1, _ = inverse_iteration(A)
    # print(g_1, _)

    # U = np.linalg.cholesky(A, upper=True)

    # print(np.matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim)))
    eyeminus = eye(dim) - matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim))

    # print(L @ L.T) # A matr

    for i in range(max_iter):
        x_norm = norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / x_norm  # +- eigenvec

        # f = E - g_1*g_1^T
        # L*y = f * nu
        # U*x = y

        # y = solve(
        #     L,
        #     matmul(eyeminus, nu),
        # )
        # x_next = solve(U, y)

        x_next = solve_bc_system(B, C, eyeminus@nu)

        # print(g_1.reshape(dim, -1).shape, g_1.reshape(-1, dim).shape)
        # x_next = matmul(
        #     matmul(
        #         np.linalg.inv(A),
        #         np.eye(dim) - np.matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim)),
        #     ),
        #     nu,
        # )

        sigma_prev = sigma
        sigma = matmul(nu, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance:
          # print(i)
          return nu, 1 / sigma, i

        sigma_prev = sigma
        x_prev = x_next

    return nu, 1 / sigma, max_iter


In [20]:
print(inverse_iteration_with_shift_one(test_hh))

(array([ 0.6536,  0.6096, -0.4485,  0.0068]), np.float64(0.3980596964013644), 45)


In [21]:
print(np.linalg.eig(test_hh))

EigResult(eigenvalues=array([0.9683, 0.3981, 0.5503, 0.0158]), eigenvectors=array([[ 0.7224, -0.6536,  0.2253, -0.008 ],
       [-0.3263, -0.6096, -0.7224, -0.0075],
       [ 0.6096,  0.4485, -0.6536, -0.0068],
       [ 0.0075, -0.0068, -0.008 ,  0.9999]]))


## Testing

In [24]:
def TEST_generate_householder_mat(dim, lambda_range):
    rng = np.random.default_rng()

    # w = rng.uniform(-lambda_range, lambda_range, dim)
    w = rng.random(dim)
    w = w / norm(w, ord=None)

    lambdas = rng.uniform(high=lambda_range, size=dim)
    # lambdas = rng.random(dim)
    lambda_mat = eye(dim)

    for i in range(dim):
        lambda_mat[i, i] = lambdas[i]
    # print(lambdas, "\n", lambda_mat)

    idx_second_min = np.where(lambdas == np.sort(lambdas)[1])
    # print(idx_second_min[0])
    # second min
    # second_lam = np.sort(lambdas)[1]
    # print(second_lam)

    H = np.eye(dim) - 2 * matmul(w.reshape(dim, -1), w.reshape(-1, dim))

    # second_g = H[:, 1]
    # print(second_g)
    # print(lambdas)
    # print(H)
    # print(H)
    # print(lambdas)
    return (
        matmul(matmul(H, lambda_mat), H.T),
        lambdas[idx_second_min[0]],
        H[:, idx_second_min[0]],
    )


def calculate_accuracy_measure(
    A: np.ndarray, x: np.ndarray, lambda_val: float
) -> float:
    """
    Calculate accuracy measure r = ||Ax - λx||_∞

    Args:
        A: Input matrix
        x: Eigenvector
        lambda_val: Eigenvalue
    Returns:
        r: Accuracy measure (max absolute residual)
    """
    # Calculate residual vector r = Ax - λx
    Ax = matmul(A, x)
    lambda_x = lambda_val * x
    residual = Ax - lambda_x

    # Find max absolute value
    r = np.max(np.abs(residual))

    return r


def TEST(size_range, lambda_range, lambda_diff=10 ** (-5)):
    rng = np.random.default_rng()
    cnt = 0

    lambdas_diffs = []
    vectors_diffs = []
    iterations = []

    while cnt < 10:
        size = rng.integers(2, size_range+1)

        try:
            A, eigenval, eigenvec = TEST_generate_householder_mat(size, lambda_range)

            vec, val, iter = inverse_iteration_with_shift_one(A, tolerance=lambda_diff)

            # print("calculated: \n", vec, "\n", val)
            # print()
            # print("actual: \n", eigenvec.ravel(), "\n", eigenval)
            # print("--------------")

            cnt += 1
            # print(vec, eigenvec.reshape(-1))
            vectors_diffs.append(
                min(
                    abs(angle_between(vec, eigenvec.reshape(-1))),
                    abs(angle_between(-vec, eigenvec.reshape(-1))),
                ),
            )
            lambdas_diffs.append(abs(val - eigenval))
            iterations.append(iter)
        except np.linalg.LinAlgError:
            print("except")

    print(
        size_range,
        " -- ",
        lambda_range,
        " -- ",
        f"eps={lambda_diff}",
        " -- ",
        f"{np.mean(lambdas_diffs)}",
        " -- ",
        np.mean(vectors_diffs),
        " -- ",
        calculate_accuracy_measure(A, vec, val),
        " -- ",
        np.mean(iterations),
    )

for size_range in [10, 30, 50]:
  # print(size_range)
  for eigenvals in [2, 50]:
    # print(eigenvals)
    for eig_diffs in [10**(-10), 10**(-13)]:
      # print(eig_diffs)
      TEST(size_range, eigenvals, eig_diffs)
  
# TEST(10, 2)


10  --  2  --  eps=1e-10  --  4.145499220341853e-05  --  0.09471080381744294  --  2.1676646545440437e-07  --  94.3
10  --  2  --  eps=1e-13  --  5.757061494193749e-14  --  7.720014036938479e-07  --  9.982293465785053e-08  --  50.2
10  --  50  --  eps=1e-10  --  5.631555310969816e-08  --  6.510109004693508e-05  --  0.0008565533832562977  --  22.7
10  --  50  --  eps=1e-13  --  3.0685242569106206e-09  --  2.7275256288662177e-05  --  1.0906334025895248e-06  --  88.0
30  --  2  --  eps=1e-10  --  0.0001041531611367781  --  0.07597889696187503  --  4.715215004124501e-07  --  40.8
30  --  2  --  eps=1e-13  --  2.920682046214651e-09  --  0.0004431950370938559  --  5.49060024182535e-09  --  152.8
30  --  50  --  eps=1e-10  --  0.0006597157928904917  --  0.07449780387836825  --  4.890930194544063e-05  --  91.9
30  --  50  --  eps=1e-13  --  7.897371645526618e-12  --  3.309376276570088e-06  --  1.261740624786123e-06  --  64.8
50  --  2  --  eps=1e-10  --  2.171090789787611e-11  --  2.14747847564